Python Jupyter Notebook that combines two previously separated algorithm for:
1. Taking imported raw polls and converting to vote shares for every candidate in every constituency
2. Analysing election results for a range of specified dates

In [1]:
# Algorithm details
AlgorithmName = "ComboPollElectionAlogrithm"
AlgorithmVersion = "1_0"
AlgorithmDate = "20240524"

# Version 0_1 initial test combination of version 1_1 of the election analysis algorithm and version 3_1 of the poll analysis algorithm
# Version 0_2 incorporates a loop around different analysis dates and the ability to delete analyses if they already exist for a particular date
# Version 0_3 includes a check to use as the analysis start date: the next date after the last date analysed in the database OR the date of the earliest poll to added, whichever is sooner
# Version 0_4 check of validity periods no incorporated into the overall loop so it changes for each analysis date
# Version 1_0 First full version after testing previous incorporations

# Construct PollAnalysisAlogrithm string
ElectionAnalysisAlgorithm = AlgorithmName + "_" + AlgorithmVersion + "_" + AlgorithmDate
PollAnalysisAlgorithm = AlgorithmName + "_" + AlgorithmVersion + "_" + AlgorithmDate

In [2]:
# Import required modules
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib

In [3]:
# Set core variables for the rest of the algorithm
ElectionDate = datetime.date(2024, 7, 4) # Set the date of the election

AutoStartDateFlag = True # if true, use the next date after the last date analysed in the database
# OR the date of the earliest poll to added, whichever is sooner
AutoEndDateFlag = True # if true, use the date of the last poll

ManualStartDate = datetime.date(2024, 1, 1) # start date of election analyses if auto is not used
ManualEndDate = datetime.date(2024, 5, 23) # end date of election analyses if auto is not used

DeleteAnalysesFlag = True # if true, already existing analyses will be delete

DetailedRankThreshold = 5 # Variable for identifying from a poll's rank, whether it is a 'national' or 'detailed' poll

In [4]:
#Connect to database 'UK_General_Election' using SQlAlchemy
connection_str = "DRIVER={SQL SERVER};SERVER=DANZPOOTA;DATABASE=UK_General_Election;TRUSTED_CONNECTION=YES"
params = urllib.parse.quote_plus(connection_str)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)
conn = engine.connect()

In [5]:
# Determine the start and end date for the election analysis loop
if AutoStartDateFlag == False:
    StartDate = ManualStartDate
else:
    LastAnalysisDateQuery = "SELECT TOP(1) ElectionPredictionDate FROM ElectionPredictionMeta ORDER BY ElectionPredictionDate DESC"
    LastAnalysisDateList = [i[0] for i in engine.execute(LastAnalysisDateQuery)]
    LastAnalysisDate = LastAnalysisDateList[0]
    
    EarliestNewPollQuery = "SELECT TOP(1) pm.PollDate from PollMeta AS pm LEFT JOIN PollAnalysisMeta AS pam ON pm.PollID = pam.PollID WHERE pam.PollID IS NULL ORDER BY pm.PollDate ASC"
    EarliestNewPollList = [i[0] for i in engine.execute(EarliestNewPollQuery)]
    EarliestNewPollDate = EarliestNewPollList[0]
    
    if LastAnalysisDate > EarliestNewPollDate:
        StartDate = pd.to_datetime(EarliestNewPollDate).date()
    else:    
        StartDate = pd.to_datetime(LastAnalysisDate).date() + timedelta(days=1)
    
if AutoEndDateFlag == False:
    EndDate = ManualEndDate
else:
    LastPollDateQuery = """SELECT TOP(1) pm.PollDate FROM PollAnalysisMeta AS pam
    INNER JOIN PollMeta AS pm ON pm.PollID = pam.PollID
    INNER JOIN RegionTypes AS rt ON rt.RegionType = pm.PollType
    ORDER BY pm.PollDate DESC"""
    LastPollDateList = [i[0] for i in engine.execute(LastPollDateQuery)]
    LastPollDate = LastPollDateList[0]
    EndDate = pd.to_datetime(LastPollDate).date()
    
print(StartDate)
print(EndDate)

2024-01-01
2024-05-23


In [6]:
# Determine validity of polls
# Move to the main elections analysis loop
ElectionPredictionDate = StartDate

DaysToElection = ElectionDate - ElectionPredictionDate
if DaysToElection.days < 31:
    NationalValidPeriod = 7
    DetailedValidPeriod = 30
else:
    NationalValidPeriod = 30
    DetailedValidPeriod = 90

In [7]:
# Get list of unanalysed polls to the loop around
PollsListQuery = "SELECT A.PollID from PollMeta A LEFT JOIN PollAnalysisMeta B ON A.PollID = B.PollID WHERE B.PollID IS NULL"
PollsList = [i[0] for i in engine.execute(PollsListQuery)]
PollsList

[]

In [8]:
%%time

# Main algorithm loop to convert poll vote shres into candidate vote shares
for PollID in PollsList:
    
    # Get the Poll Meta data from the database    
    PollMetaQuery = "SELECT * From PollMeta WHERE PollID = '<PollID>'"
    PollMetaQuery = PollMetaQuery.replace("<PollID>",PollID)
    PollMeta_df = pd.read_sql(PollMetaQuery,conn)
    
    # Get key variables for polls
    PollType = PollMeta_df.at[0,'PollType']
    PollScope = PollMeta_df.at[0,'PollScope']    
    
    #Query for generating list of applicable poll regions
    AllRegionsQuery = "SELECT RegionName FROM RegionRegionTypes WHERE RegionType = '<PollType>'"
    
    # Get from database all of the region vote shares relating to the PollID
    PollSharesQuery = "SELECT PollDetailsID, RegionName, Party, VoteShare AS PollShare FROM PollDetails WHERE PollID = '<PollID>' ORDER BY RegionName, Party"
    PollSharesQuery = PollSharesQuery.replace("<PollID>",PollID)
    PollShares_df = pd.read_sql(PollSharesQuery,conn)
    
    # Check to see if it is a constituency only poll as this needs to be handled very differently
    # The alogrithm cannot be allowed to look for region shares as it will not find any for a constituency poll
    if PollType == "Constituency":
        ConstituencyPollFlag = True

        PreviousRegionSharesQuery = "SELECT CandidateID, Constituency, Party, PreviousShare FROM Candidates WHERE Constituency = '<Constituency>'"
        PreviousRegionSharesQuery = PreviousRegionSharesQuery.replace("<Constituency>",PollScope)
        PreviousRegionShares_df = pd.read_sql(PreviousRegionSharesQuery,conn)
        
        ConstituencyPollRegion = PollShares_df.at[0,'RegionName']
        
        PreviousRegionShares_df['RegionParty'] = ConstituencyPollRegion + PreviousRegionShares_df['Party']
        PollShares_df['RegionParty'] = PollShares_df['RegionName'] + PollShares_df['Party']
        
    else:
        ConstituencyPollFlag = False
        
        # Generate a list of all of the regions applicable to a poll when it is not a constituency one
        if PollScope == "All":
            AllRegionsQuery = AllRegionsQuery.replace("<PollType>",PollType)
            RegionsList = [i[0] for i in engine.execute(AllRegionsQuery)]
        else:
            RegionsList = [PollScope]     

        # Query for the region vote shares from the previous election
        PreviousRegionSharesQuery = """SELECT r.RegionName, can.Party, SUM(can.PreviousVotes) AS 'TotalVotes',
        CAST(SUM(can.PreviousVotes) AS FLOAT) / SUM(SUM(can.PreviousVotes)) OVER() AS 'RawPreviousShare',
        SUM(can.PreviousStanding) AS 'PreviousCandidates',
        SUM(can.CurrentStanding) AS 'CurrentCandidates'
        FROM Candidates AS can
        INNER JOIN Constituencies as con ON con.ConstituencyName = can.Constituency
        INNER JOIN RegionConstituencies AS rc ON  rc.ConstituencyName = con.ConstituencyName
        INNER JOIN Regions AS r ON r.RegionName = rc.RegionName
        INNER JOIN RegionRegionTypes AS rrt ON rrt.RegionName = r.RegionName
        WHERE r.RegionName = '<RegionName>' AND rrt.RegionType = '<RegionType>'
        GROUP BY r.RegionName, can.Party
        ORDER BY r.RegionName, can.Party"""

        PreviousRegionSharesQuery = PreviousRegionSharesQuery.replace("<RegionType>",PollType)

        # Get from database all of the vote shares for the poll regions from the previous election
        PreviousRegionShares_df = pd.DataFrame(columns=["RegionName","Party","TotalVotes","RawPreviousShare","PreviousCandidates","CurrentCandidates"])

        # Loop through all regions applicable to this particular poll
        for Region in RegionsList:
            # Run query for the raw previous region shares
            ModRegionSharesQuery = PreviousRegionSharesQuery.replace("<RegionName>",Region)
            IndRegionShares_df = pd.read_sql(ModRegionSharesQuery,conn)  

            # Join this particular region's numbers to the overall dataframe for this poll
            PreviousRegionShares_df = pd.concat([PreviousRegionShares_df,IndRegionShares_df],axis=0)

        # Reset the index column
        PreviousRegionShares_df.reset_index(drop=True,inplace=True)

        # Replace all 0 values with 1000 to ensure the new vote share can be calculated without dividing by zero
        PreviousRegionShares_df['PreviousCandidates'].replace(0,1000,inplace=True)

        #Calculated the adjusted previous share based on the number of candidates actually standing
        PreviousRegionShares_df['PreviousShare'] = PreviousRegionShares_df['RawPreviousShare'] * PreviousRegionShares_df['CurrentCandidates']/PreviousRegionShares_df['PreviousCandidates']

        # Revert previous candidates back to zero
        PreviousRegionShares_df['PreviousCandidates'].replace(1000,0,inplace=True)
    
    
        # Create a column to join the two dataframes
        PollShares_df['RegionParty'] = PollShares_df['RegionName'] + PollShares_df['Party']
        PreviousRegionShares_df['RegionParty'] = PreviousRegionShares_df['RegionName'] + PreviousRegionShares_df['Party']
    
    # Exit first constituency poll if statement
    # Join the dataframes on 'RegionParty'
    PreviousRegionShares_df = PreviousRegionShares_df.merge(PollShares_df[['PollDetailsID','RegionParty','PollShare']], how='left', on='RegionParty')

    # Determine swing
    PreviousRegionShares_df['Swing'] = PreviousRegionShares_df['PollShare'] - PreviousRegionShares_df['PreviousShare']

    # Check if it's a constituency poll
    if ConstituencyPollFlag == True:
        ConstituencyShares_df = PreviousRegionShares_df.copy()
        ConstituencyShares_df['RegionName'] = ConstituencyShares_df['Constituency']
        
    else:
        # Query for pulling out previous election shares for applicable constituencies
        CandidatesQuery = """SELECT can.CandidateID, r.RegionName, can.Constituency, can.Party, can.PreviousShare
        FROM Candidates AS can
        INNER JOIN Constituencies as con ON con.ConstituencyName = can.Constituency
        INNER JOIN RegionConstituencies AS rc ON  rc.ConstituencyName = con.ConstituencyName
        INNER JOIN Regions AS r ON r.RegionName = rc.RegionName
        INNER JOIN RegionRegionTypes AS rrt ON rrt.RegionName = r.RegionName
        WHERE r.RegionName = '<RegionName>' AND rrt.RegionType = '<RegionType>'
        AND can.CurrentStanding = 1
        ORDER BY can.Constituency, can.Party"""

        CandidatesQuery = CandidatesQuery.replace("<RegionType>",PollType)

        # Calculate the swings for every candidate in every constituency
        ConstituencyShares_df = pd.DataFrame(columns=["CandidateID","RegionName","Constituency","Party","PreviousShare","NewShareRaw","VoteShare"])
        IndConstituencyShares_df = pd.DataFrame(columns=["CandidateID","RegionName","Constituency","Party","PreviousShare","NewShareRaw","VoteShare"])

        # Need to cycle through every region to pull out the candidates for each in turn, then get the swing for each
        for Region in RegionsList:

            ModCandidatesQuery = CandidatesQuery.replace("<RegionName>",Region)
            IndConstituencyShares_df = pd.read_sql(ModCandidatesQuery,conn)   

            ConstituencyShares_df = pd.concat([ConstituencyShares_df,IndConstituencyShares_df],axis=0)

        ConstituencyShares_df.reset_index(drop=True,inplace=True)
    
        # Create 'RegionParty' to allow merging with the previous region shares dataframe
        ConstituencyShares_df['RegionParty'] = ConstituencyShares_df['RegionName'] + ConstituencyShares_df['Party']
        
        # Join the dataframes on 'RegionParty'
        ConstituencyShares_df = ConstituencyShares_df.merge(PreviousRegionShares_df[['RegionParty','Swing']], how='left', on='RegionParty')

    # Exit of 2nd constituency poll IF statement    
    # Ensure the datatypes are numeric of the columns to be used in the calculation
    ConstituencyShares_df["PreviousShare"] = pd.to_numeric(ConstituencyShares_df["PreviousShare"])
    ConstituencyShares_df["Swing"] = pd.to_numeric(ConstituencyShares_df["Swing"])

    ConstituencyShares_df["NewShareRaw"] = ConstituencyShares_df["PreviousShare"] + ConstituencyShares_df["Swing"] 
    ConstituencyShares_df["NewShareRaw"] = np.where(ConstituencyShares_df["NewShareRaw"] < 0, 0,ConstituencyShares_df["NewShareRaw"])

    # Determine the factor needed to ensure vote shares for each constituency sum to 1
    ConstituencyShares_df['ConstRawShareTotals'] = ConstituencyShares_df['NewShareRaw'].groupby(ConstituencyShares_df['Constituency']).transform('sum')

    # Modify the raw vote shares to ensure they sum to 1
    ConstituencyShares_df['VoteShare'] = ConstituencyShares_df['NewShareRaw']/ConstituencyShares_df['ConstRawShareTotals']

    ConstituencyShares_df['VoteShareCheck'] = ConstituencyShares_df['VoteShare'].groupby(ConstituencyShares_df['Constituency']).transform('sum')

    # Create PollAnalysisMeta details for inserting into database
    PollAnalysisMeta_df = pd.DataFrame(columns=["PollID","PollAnalysisDate","PollAnalysisAlgorithm"])

    PollAnalysisMeta_df.at[0,"PollID"] = PollID
    PollAnalysisMeta_df.at[0,"PollAnalysisAlgorithm"] = PollAnalysisAlgorithm

    # The date of the analysis is always today's date
    PollAnalysisMeta_df.at[0,"PollAnalysisDate"] = datetime.date.today()

    PollAnalysisMeta_df.to_sql('PollAnalysisMeta', conn, if_exists='append', index=False)

    # Initial poll analysis values are now inserted into the database to allow these to be queried for the constituency shares
    PollAnalysisRegions_df = PreviousRegionShares_df[['PollDetailsID','Swing']].copy()

    # Get the recently inserted PollAnalysis ID from the database
    PollAnalysisIDQuery = "SELECT PollAnalysisID FROM PollAnalysisMeta WHERE PollID = '<PollID>'"
    PollAnalysisIDQuery = PollAnalysisIDQuery.replace("<PollID>",PollID)

    PollAnalysisID = [i[0] for i in engine.execute(PollAnalysisIDQuery)][0]
    PollAnalysisRegions_df['PollAnalysisID'] = PollAnalysisID

    PollAnalysisRegions_df.to_sql('PollAnalysisRegions', conn, if_exists='append', index=False)
    
    # Create the dataframe for insertion into the database and insert
    ConstituencyShares_df['PollAnalysisRegionID'] = PollID + ConstituencyShares_df['RegionParty'] + PollAnalysisID
    PollAnalysisConstituencies_df = ConstituencyShares_df[['PollAnalysisRegionID','CandidateID','VoteShare']]
    PollAnalysisConstituencies_df.to_sql('PollAnalysisConstituencies', conn, if_exists='append', index=False)

CPU times: total: 0 ns
Wall time: 0 ns


Separation between the previoulsy segragated poll analysis and election analysis algorithms

In [9]:
DeleteAnalysisQueryRaw = """DELETE eppu FROM ElectionPredictionPollsUsed AS eppu
INNER JOIN ElectionPredictionMeta AS epm ON epm.ElectionPredictionID = eppu.ElectionPredictionID
WHERE epm.ElectionPredictionDate = '<ElectionPredictionDate>'

DELETE epc FROM ElectionPredictionCandidates AS epc
INNER JOIN ElectionPredictionMeta AS epm ON epm.ElectionPredictionID = epc.ElectionPredictionID
WHERE epm.ElectionPredictionDate = '<ElectionPredictionDate>'

DELETE epcon FROM ElectionPredictionConstituencies AS epcon
INNER JOIN ElectionPredictionMeta AS epm ON epm.ElectionPredictionID = epcon.ElectionPredictionID
WHERE epm.ElectionPredictionDate = '<ElectionPredictionDate>'

DELETE epo FROM ElectionPredictionOverall AS epo
INNER JOIN ElectionPredictionMeta AS epm ON epm.ElectionPredictionID = epo.ElectionPredictionID
WHERE epm.ElectionPredictionDate = '<ElectionPredictionDate>'

DELETE FROM ElectionPredictionMeta WHERE ElectionPredictionDate = '<ElectionPredictionDate>'"""

In [10]:
%%time
# Combined Election Analysis

# Setup variables for the loop
ElectionPredictionDate = StartDate
Delta = timedelta(days=1)

# Loop around all of the analysis dates
while ElectionPredictionDate <= EndDate:    
    #print(ElectionPredictionDate)
        
    DaysToElection = ElectionDate - ElectionPredictionDate
    if DaysToElection.days < 31:
        NationalValidPeriod = 7
        DetailedValidPeriod = 30
    else:
        NationalValidPeriod = 30
        DetailedValidPeriod = 90
    
    #print("Prediction Date:",ElectionPredictionDate,"Days To Election:",DaysToElection.days,"NationalValidPeriod:",NationalValidPeriod)
    
    #Check if there is already an analysis for this analysis date
    ElectionPredictionDateStr = datetime.datetime.strftime(ElectionPredictionDate, '%Y%m%d')
    AnalysisExistsQuery = "SELECT Count(ElectionPredictionID) FROM ElectionPredictionMeta WHERE ElectionPredictionDate = '<ElectionPredictionDate>'"
    AnalysisExistsQuery = AnalysisExistsQuery.replace('<ElectionPredictionDate>',ElectionPredictionDateStr)
    AnalysisExistsList = [i[0] for i in engine.execute(AnalysisExistsQuery)]
    AnalysisExistsInt = AnalysisExistsList[0]
    #print(AnalysisExistsInt)

    if AnalysisExistsInt < 1 or DeleteAnalysesFlag == True:

        if AnalysisExistsInt >=1:
            # Delete the analysis
            DeleteAnalysisQuery = DeleteAnalysisQueryRaw.replace('<ElectionPredictionDate>',ElectionPredictionDateStr)
            engine.execute(DeleteAnalysisQuery)
    
        # Get the list of polls that have actually been analysed and incorporated into the database
        AnalysedPollsQuery = """SELECT pam.PollID, pm.Pollster, pm.PollType, pm.PollScope, pm.PollDate, rt.RegionTypeRank  FROM PollAnalysisMeta AS pam
        INNER JOIN PollMeta AS pm ON pm.PollID = pam.PollID
        INNER JOIN RegionTypes AS rt ON rt.RegionType = pm.PollType"""

        AnalysedPolls_df = pd.read_sql(AnalysedPollsQuery,conn)

        # Add prediction date and convert to datetime date so that it can be used in a calcualtion
        AnalysedPolls_df['PredictionDate'] = ElectionPredictionDate
        AnalysedPolls_df['PredictionDate'] = pd.to_datetime(AnalysedPolls_df['PredictionDate'])

        # Convert the date column to datetime type
        AnalysedPolls_df['PollDate'] = pd.to_datetime(AnalysedPolls_df['PollDate'])

        # Determine how many days from the prediction date a poll was taken
        AnalysedPolls_df['DateDelta'] = AnalysedPolls_df['PredictionDate'] - AnalysedPolls_df['PollDate']

        # Assign a rank to each poll
        AnalysedPolls_df['PollRank'] = np.where(AnalysedPolls_df['PollScope']=='All',AnalysedPolls_df['RegionTypeRank'],AnalysedPolls_df['RegionTypeRank']-1)

        # Determine the applicability of each poll based on the day delta
        AnalysedPolls_df['PollApplicability'] = np.where((AnalysedPolls_df['DateDelta'] < pd.Timedelta(DetailedValidPeriod, unit="d")) \
                                                         & (AnalysedPolls_df['DateDelta'] >= pd.Timedelta(0, unit="d")) \
                                                         & (AnalysedPolls_df['PollRank'] <= DetailedRankThreshold),1, \
                                                         np.where((AnalysedPolls_df['DateDelta'] < pd.Timedelta(NationalValidPeriod, unit="d")) \
                                                         & (AnalysedPolls_df['DateDelta'] >= pd.Timedelta(0, unit="d")) \
                                                         & (AnalysedPolls_df['PollRank'] > DetailedRankThreshold),1,0))

        # Copy the polls that fall within the date delta to a new data frame
        ApplicablePolls_df = AnalysedPolls_df[AnalysedPolls_df['PollApplicability'] == 1].copy()
        ApplicablePolls_df.reset_index(drop=True,inplace=True)

        # Check for duplicate polls of the same type
        # Create column that would contain the info to check for duplicates
        ApplicablePolls_df['DuplicateCheck'] = ApplicablePolls_df['Pollster'] + ApplicablePolls_df['PollType'] + ApplicablePolls_df['PollScope'] 

        # The dataframe needs to be sorted to ensure the latest poll is kept and older duplicates are removed
        ApplicablePolls_df.sort_values(by='DateDelta',inplace=True)

        # Drop duplicates
        ApplicablePolls_df.drop_duplicates(subset=['DuplicateCheck'],inplace=True)

        # Reset the dataframe index after being sorted and duplicates removed
        ApplicablePolls_df.reset_index(drop=True,inplace=True)

        # Main algorithm to create resultant candidate shares
        # Create dataframe with every candidate as rows
        SelectCandidatesQuery = "SELECT can.CandidateID, can.Constituency, can.Party, can.PreviousShare FROM Candidates AS can WHERE can.CurrentStanding = 1"
        Candidates_df = pd.read_sql(SelectCandidatesQuery,conn)

        # Descending Loop around every rank
        # start point = max rank from applicable polls df
        MaxRank = ApplicablePolls_df['PollRank'].max()
        MinRank = ApplicablePolls_df['PollRank'].min()

        # Boolean to record if this is the first run through for the running total
        FirstRun = True

        for Rank in range(MaxRank,MinRank-1,-1):

            # Create df from all polls relating to the current rank
            # First create list of applicable polls
            RankApplicablePolls_df = ApplicablePolls_df[ApplicablePolls_df['PollRank'] == Rank].copy()
            RankApplicablePolls = RankApplicablePolls_df['PollID'].tolist()

            for PollID in RankApplicablePolls:
                # print(Rank,":",PollID)        

                # Query for getting the poll analysis details
                SelectPollDetailsQuery = """SELECT pac.CandidateID, pac.VoteShare AS '<PollID>' FROM PollAnalysisConstituencies AS pac
                INNER JOIN PollAnalysisRegions AS par on par.PollAnalysisRegionID = pac.PollAnalysisRegionID
                INNER JOIN PollAnalysisMeta AS pam ON pam.PollAnalysisID = par.PollAnalysisID
                WHERE pam.PollID = '<PollID>'
                ORDER BY pac.CandidateID"""

                # Insert the PollID into the query
                SelectPollDetailsQuery = SelectPollDetailsQuery.replace("<PollID>",PollID)

                # Pull query into a data frame for the results
                PollAnalysis_df = pd.read_sql(SelectPollDetailsQuery,conn)

                # Merge with Candidates_df
                # PreviousRegionShares_df = PreviousRegionShares_df.merge(PollRegionShares_df[['PollDetailsID','RegionParty','PollShare']], how='left', on='RegionParty')
                Candidates_df = Candidates_df.merge(PollAnalysis_df, how='left', on='CandidateID')

            # Average the columns for each row (candidate) for this rank
            if RankApplicablePolls != []:
                if FirstRun == True:
                    Candidates_df[Rank] = Candidates_df[RankApplicablePolls].mean(axis=1)
                    # Set the first run to false now that the loop has been gone though
                    FirstRun = False
                else:
                    RankApplicablePolls.append(PreviousRank)
                    Candidates_df[Rank] = Candidates_df[RankApplicablePolls].mean(axis=1)

                # Record the last rank to be evaluated        
                PreviousRank = Rank

        Candidates_df['VoteShareRaw'] = Candidates_df[Rank].copy()

        # Determine the factor needed to ensure vote shares for each constituency sum to 1
        Candidates_df['ConstRawShareTotals'] = Candidates_df['VoteShareRaw'].groupby(Candidates_df['Constituency']).transform('sum')

        # Modify the raw vote shares to ensure they sum to 1
        Candidates_df['VoteShare'] = Candidates_df['VoteShareRaw']/Candidates_df['ConstRawShareTotals']

        Candidates_df['VoteShareCheck'] = Candidates_df['VoteShare'].groupby(Candidates_df['Constituency']).transform('sum')

        Candidates_df['VoteShareChange'] = Candidates_df['VoteShare'] - Candidates_df['PreviousShare']

        # Determine the constituency winners
        # https://stackoverflow.com/questions/15705630/get-the-rows-which-have-the-max-value-in-groups-using-groupby
        #df.sort_values('count').drop_duplicates(['Sp', 'Mt'], keep='last')
        ConstWinners_df = Candidates_df[['Constituency','Party','VoteShare','VoteShareChange']].copy()
        ConstWinners_df = ConstWinners_df.sort_values(['Constituency','VoteShare'],ascending = [True, False])
        ConstWinners_df['Majority'] = ConstWinners_df['VoteShare'] - ConstWinners_df['VoteShare'].shift(-1)
        ConstWinners_df['SecondParty'] = ConstWinners_df['Party'].shift(-1)

        # Calculate the swing for each winner
        ConstWinners_df['Swing'] = (ConstWinners_df['VoteShareChange'] - ConstWinners_df['VoteShareChange'].shift(-1))/2

        # Keep only the winners of each constituency
        ConstWinners_df = ConstWinners_df.sort_values('VoteShare').drop_duplicates(['Constituency'], keep='last')
        ConstWinners_df = ConstWinners_df.sort_values('Constituency')

        ConstWinners_df.reset_index(drop=True,inplace=True)

        # Determine how many constituencies each party has won
        OverallResults_df = ConstWinners_df[['Party','Constituency']].groupby('Party').count()
        OverallResults_df.reset_index(drop=False,inplace=True)

        # Determine total vote shares for each party
        OverallVoteShare_df = Candidates_df[['Party','VoteShare']].copy()
        OverallVoteShare_df['PartyShare'] = OverallVoteShare_df['VoteShare'].groupby(OverallVoteShare_df['Party']).transform('sum')
        OverallVoteShare_df = OverallVoteShare_df.drop_duplicates(['Party'], keep='last')
        OverallVoteShare_df.drop(['VoteShare'], axis=1, inplace=True)
        OverallVoteShare_df.sort_values(by='PartyShare', ascending=False, inplace=True)
        OverallVoteShare_df['PartyShare'] = OverallVoteShare_df['PartyShare']/OverallVoteShare_df['PartyShare'].sum()
        OverallVoteShare_df.reset_index(drop=True,inplace=True)

        # Combine the overall vote shares and constituency numbers into one dataframe
        Overall_df = OverallVoteShare_df.merge(OverallResults_df, how='left', on='Party')
        Overall_df = Overall_df.fillna(0)
        Overall_df.sort_values(by='Constituency', ascending=False, inplace=True)
        Overall_df.reset_index(drop=True,inplace=True)

        # Election prediction analysis date is today
        ElectionAnalysisDate = datetime.date.today()

        # Create the election prediction ID for the database
        ElectionPredictionID = datetime.datetime.strftime(ElectionAnalysisDate, '%Y%m%d') + datetime.datetime.strftime(ElectionPredictionDate, '%Y%m%d') + ElectionAnalysisAlgorithm

        # Create the ElectionPredictionMeta table content for this analysis
        ElectionPredictionMeta_df = pd.DataFrame(columns=["ElectionAnalysisDate","ElectionPredictionDate","ElectionAnalysisAlgorithm"])
        ElectionPredictionMeta_df.at[0,"ElectionAnalysisDate"] = ElectionAnalysisDate
        ElectionPredictionMeta_df.at[0,"ElectionPredictionDate"] = ElectionPredictionDate
        ElectionPredictionMeta_df.at[0,"ElectionAnalysisAlgorithm"] = ElectionAnalysisAlgorithm

        # Create the ElectionPredictionPolls used content
        ElectionPredictionPollsUsed_df = pd.DataFrame(columns=['ElectionPredictionID','PollID'])
        ElectionPredictionPollsUsed_df['PollID'] = ApplicablePolls_df['PollID'].copy()
        ElectionPredictionPollsUsed_df['ElectionPredictionID'] = ElectionPredictionID

        # Create ElectionPredictionCandidates data
        ElectionPredictionCandidates_df = pd.DataFrame(columns=['ElectionPredictionID','CandidateID','VoteShare'])
        ElectionPredictionCandidates_df[['CandidateID','VoteShare']] = Candidates_df[['CandidateID','VoteShare']].copy()
        ElectionPredictionCandidates_df['ElectionPredictionID'] = ElectionPredictionID

        # Create ElectionPredictionConstituencies data
        ElectionPredictionConstituencies_df = pd.DataFrame(columns=['ElectionPredictionID','Constituency','WinningParty','SecondParty','VoteShare','Majority','GAIN','LOSS','Swing'])
        ElectionPredictionConstituencies_df[['Constituency','WinningParty','SecondParty','VoteShare','Majority','Swing']] = ConstWinners_df[['Constituency','Party','SecondParty','VoteShare','Majority','Swing']].copy()

        PreviousWinnersQuery = "SELECT ConstituencyName AS 'Constituency', FirstParty AS 'PreviousWinner' FROM Constituencies"
        PreviousWinners_df = pd.read_sql(PreviousWinnersQuery,conn)
        ElectionPredictionConstituencies_df = ElectionPredictionConstituencies_df.merge(PreviousWinners_df, how='left', on='Constituency')
        ElectionPredictionConstituencies_df['GAIN'] = np.where(ElectionPredictionConstituencies_df['WinningParty'] == ElectionPredictionConstituencies_df['PreviousWinner'],ElectionPredictionConstituencies_df['WinningParty']+" HOLD",ElectionPredictionConstituencies_df['WinningParty']+" GAIN")
        ElectionPredictionConstituencies_df['LOSS'] = np.where(ElectionPredictionConstituencies_df['WinningParty'] == ElectionPredictionConstituencies_df['PreviousWinner'],ElectionPredictionConstituencies_df['PreviousWinner']+" HOLD",ElectionPredictionConstituencies_df['PreviousWinner']+" LOSS")
        ElectionPredictionConstituencies_df['ElectionPredictionID'] = ElectionPredictionID

        # Create ElectionPredictionOverall data
        ElectionPredictionOverall_df = pd.DataFrame(columns=['ElectionPredictionID','Party','VoteShare','Constituencies'])
        ElectionPredictionOverall_df[['Party','VoteShare','Constituencies']] = Overall_df[['Party','PartyShare','Constituency']].copy()
        ElectionPredictionOverall_df['Constituencies'] = ElectionPredictionOverall_df['Constituencies'].astype('int')
        ElectionPredictionOverall_df['ElectionPredictionID'] = ElectionPredictionID

        # Insert data into the database
        ElectionPredictionMeta_df.to_sql('ElectionPredictionMeta', conn, if_exists='append', index=False)
        ElectionPredictionPollsUsed_df.to_sql('ElectionPredictionPollsUsed', conn, if_exists='append', index=False)
        ElectionPredictionCandidates_df.to_sql('ElectionPredictionCandidates', conn, if_exists='append', index=False)
        ElectionPredictionConstituencies_df.to_sql('ElectionPredictionConstituencies', conn, if_exists='append', index=False)
        ElectionPredictionOverall_df.to_sql('ElectionPredictionOverall', conn, if_exists='append', index=False)
    
    ElectionPredictionDate += Delta # Increment ElectionPredictionDate

CPU times: total: 38.6 s
Wall time: 14min 23s


In [11]:
Candidates_df.head(10)

,CandidateID,Constituency,Party,PreviousShare,20240523TechneGBAll,8,20240523More in CommonITL1Region-NINAAll,20240522SurvationGBSixAll,20240522YouGovGBSixAll,20240520DeltapollGBSixAll,...,20240326SurvationLondonFiveAll,20240302More in CommonMICBlueWallAll,4,20240419TechneConstituencyPortsmouth North,1,VoteShareRaw,ConstRawShareTotals,VoteShare,VoteShareCheck,VoteShareChange
0,Aberafan MaestegCon,Aberafan Maesteg,Con,0.226279,0.000000,0.000000,0.103941,0.140839,0.087108,0.067328,...,NaN,NaN,0.068545,NaN,0.068545,0.068545,1.0,0.068545,1.0,-0.157734
1,Aberafan MaestegGreen,Aberafan Maesteg,Green,0.015780,0.034414,0.034414,0.068041,0.060957,0.071516,0.020738,...,NaN,NaN,0.099767,NaN,0.099767,0.099767,1.0,0.099767,1.0,0.083987
2,Aberafan MaestegLab,Aberafan Maesteg,Lab,0.529208,0.639477,0.639477,0.580561,0.636022,0.540176,0.637649,...,NaN,NaN,0.557456,NaN,0.557456,0.557456,1.0,0.557456,1.0,0.028249
3,Aberafan MaestegLD,Aberafan Maesteg,LD,0.037030,0.036098,0.036098,0.079222,0.000000,0.000000,0.062920,...,NaN,NaN,0.033332,NaN,0.033332,0.033332,1.0,0.033332,1.0,-0.003698
4,Aberafan MaestegOther,Aberafan Maesteg,Other,0.016455,0.013308,0.013308,0.000669,0.000654,0.010856,0.000669,...,NaN,NaN,0.007049,NaN,0.007049,0.007049,1.0,0.007049,1.0,-0.009406
5,Aberafan MaestegPC,Aberafan Maesteg,PC,0.089841,0.093205,0.093205,0.041988,0.108421,0.038119,0.141319,...,NaN,NaN,0.054643,NaN,0.054643,0.054643,1.0,0.054643,1.0,-0.035198
6,Aberafan MaestegReform,Aberafan Maesteg,Reform,0.085406,0.183499,0.183499,0.125578,0.053108,0.252225,0.069377,...,NaN,NaN,0.179207,NaN,0.179207,0.179207,1.0,0.179207,1.0,0.093801
7,Aberdeen NorthCon,Aberdeen North,Con,0.245496,0.000000,0.000000,0.183162,0.152525,0.157544,0.145967,...,NaN,NaN,0.110433,NaN,0.110433,0.110433,1.0,0.110433,1.0,-0.135064
8,Aberdeen NorthGreen,Aberdeen North,Green,0.012209,0.032201,0.032201,0.020607,0.009559,0.085795,0.034436,...,NaN,NaN,0.027781,NaN,0.027781,0.027781,1.0,0.027781,1.0,0.015572
9,Aberdeen NorthLab,Aberdeen North,Lab,0.119796,0.246793,0.246793,0.261893,0.323848,0.279968,0.267252,...,NaN,NaN,0.333540,NaN,0.333540,0.333540,1.0,0.333540,1.0,0.213744


In [12]:
Overall_df

,Party,PartyShare,Constituency
0,Lab,0.443212,458.0
1,Con,0.234212,133.0
2,LD,0.090546,27.0
3,SNP,0.027109,8.0
4,DUP,0.006891,8.0
5,SF,0.007975,7.0
6,PC,0.003769,3.0
7,Other,0.014731,2.0
8,SDLP,0.002571,2.0
9,Green,0.059232,1.0


In [13]:
# Save the candidates dataframe to a csv for testing
#Candidates_df.to_csv('C:/Users/danmu/Documents/Elections/2024_Python/candidates.csv',encoding='utf-8')

In [14]:
#Close the connection with the database
conn.close()